In [ ]:
import pandas as pd

NOTE: the ranges for each category are assumed to be inclusie on both ends

In [ ]:
baseline_scores_file = '/Users/yasaman/UWEXP/analysis-scripts/surveys/results/baselines/baseline_scores.csv'
scale_scores = {'CES_D': 'CES_D_POST', 'BDI_II': 'BDI_II_POST', 'PSS': 'PSS_POST'} # ideally the scores are named the same as scales so this mapping is not needed
categorization_config = {
    'CES_D' : {"name" : "depressed_cesd_post", "rule": {(0, 16) : "NO", (17, 60) : "YES"}}, 
    'BDI_II' : {"name" : "depressed_bdi_post", "rule" : {(0, 13) : "NOT", (14, 19) : "LOW", (20, 28) : "MID", (29, 63) : "HIG"}},
    'PSS' : {"name": "stressed_pss_post", "rule": {(0, 19) : "NO", (20, 40) : "YES"}}
}
baseline_scores_with_univ_pid = '/Users/yasaman/UWEXP/analysis-scripts/surveys/results/baselines/baseline_scores_univ_pid.csv'
baseline_scores_with_univ_pid_categorized = '/Users/yasaman/UWEXP/analysis-scripts/surveys/results/baselines/baseline_scores_univ_pid_categorized.csv'

# participant ID and institution columns

In [ ]:
baseline_scores = pd.read_csv(baseline_scores_file)

In [ ]:
baseline_scores['institution'] = baseline_scores['PID'].apply(lambda x : x.split('_')[1])
baseline_scores.loc[:, 'PID'] = baseline_scores['PID'].apply(lambda x : x.split('_')[0])

In [ ]:
columns = list(baseline_scores.columns)
columns.remove('institution')
columns.remove('PID')
reordered_columns = ['PID', 'institution']
reordered_columns.extend(columns)
baseline_scores = baseline_scores[reordered_columns]

In [ ]:
baseline_scores.to_csv(baseline_scores_with_univ_pid, index=False)

# categorization columns

In [ ]:
def categorize(scores, categorization):
    intervals = pd.IntervalIndex.from_tuples(categorization.keys(), closed='both')
    intervals_labels = dict(zip(intervals.values, categorization.values()))
    categories = pd.cut(scores, intervals, labels=categorization.values())
    # but labels are ignored so I have to correct that
    return categories.replace(intervals_labels)

In [ ]:
for scale in scale_scores:
    baseline_scores[categorization_config[scale]['name']] = categorize(baseline_scores[scale_scores[scale]], 
                                                                       categorization_config[scale]['rule'])

In [ ]:
baseline_scores.to_csv(baseline_scores_with_univ_pid_categorized, index=False)